In [1]:
import torch
import torch.nn as nn
import sys
python_files_dir = "./python_files/" # python工具包位置
sys.path.append(python_files_dir)
import fid_score as official_fid

model_files_dir = "./model_files/" # 模型位置
sys.path.append(model_files_dir)
from CIFAR10.models import ResNet20 as ResNet
from CIFAR10.models import Rob_predictor as my_Rob_predictor

import foolbox as fb
from foolbox.attacks import LinfPGD


import pickle
import os
import numpy as np
from tqdm import tqdm
from PIL import Image  
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from scipy import spatial
import time
import PIL.Image
import copy
import eagerpy as ep

import torch
import torchvision.utils as utils
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.autograd import Variable
import random

device = torch.device("cuda:0") 
cpu = torch.device("cpu")
kd_tree_number = 20 # 通过多少个最近邻生成样本


from flask import Flask, render_template, request, jsonify
import logging
import os
import time

import sys
sys.path.append("./python_files")
import my_tools

import torch
import torch.nn as nn
import sys
import pickle
python_files_dir = "./python_files/" # python工具包位置
sys.path.append(python_files_dir)

model_files_dir = "./model_files/" # 模型位置
sys.path.append(model_files_dir)
import model_files as model_all

device_id = 0
device = torch.device("cuda:" + str(device_id)) 

In [2]:
# 定义中间变量字典
activation = {}
# 用来获取模型中间层输出的hook
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook
class RegreAttack():
    def __init__(self, rel_stepsize: float = 0.01 / 0.3, steps: int = 40, bounds = (0, 1)):
        self.rel_stepsize = rel_stepsize
        self.steps = steps
        self.bounds = bounds
        self.loss_fn = nn.MSELoss()

    def get_random_start(self, x0, epsilon: float):
        x0, restore_type = ep.astensor_(x0)
        x = x0 + ep.uniform(x0, x0.shape, -epsilon, epsilon)
        return restore_type(x)

    def normalize(self, gradients):
        return gradients.sign()

    # 将
    def project(self, x, x0, epsilon: float):
        x, x_restore_type = ep.astensor_(x)
        x0, x0_restore_type = ep.astensor_(x0)
        temp = x0 + ep.clip(x - x0, -epsilon, epsilon)
        return x_restore_type(temp)
    
    def run(self, model, inputs, epsilon: float):
        x0 = inputs.clone().detach().requires_grad_()
        _min, _max = self.bounds
        init_label = model(x0)
        # print("攻击中的init_label: ", init_label)

        # 随机添加扰动，生成新的样本
        x = self.get_random_start(x0, epsilon)
        x = ep.clip(x, _min, _max)
        x = x.clone().detach().requires_grad_()

        # 梯度下降攻击
        for _ in range(self.steps):
            model.zero_grad()  # 清空模型参数的梯度信息
            # 计算扰动后的损失
            output = model(x)
            mse_loss = self.loss_fn(init_label, output)
            mse_loss.backward(retain_graph=True)
            gradients = x.grad.data

            gradients = self.normalize(gradients)
            x = x + self.rel_stepsize * gradients * epsilon
            x = self.project(x, x0, epsilon)
            x = ep.clip(x, _min, _max)
            x.grad = None  # 清空 x 的梯度信息
            x = x.clone().detach_().requires_grad_()
        return x

# 通过对抗扰动获取图片真实鲁棒性（回归模型）
def adversarial_robustness_regre(model, image_path, label, device):
    print("获取对抗鲁棒性中.....")
    sys.stdout.flush()
    time1 = time.time()
    transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
            ]
        )
    image = Image.open(image_path)
    image = transform(image)
    image = image.unsqueeze(0).to(device)  # 增加一个维度维batch维度
    copy_model = copy.deepcopy(model)
    copy_model.eval()

    attack = RegreAttack()
    epsilon = 0.001
    flag = False
    while(flag == False):
        if epsilon >= 0.033:
            break
        adv = attack.run(copy_model, image, epsilon=epsilon)
        label2 = model(adv)[0]
        print("误差: ", abs(label2 - label))
        if abs(label2 - label) >= 0.05: # 0~1对应-80~80度，所以0.03125对应的是误差超过5度，0.05对应8度, 0.0625对应10度
            flag=True
        epsilon = epsilon + 0.001

    sys.stdout.flush()
    time2 = time.time()
    print("获取对抗鲁棒性消耗时间：", time2-time1)
    return epsilon-0.001

def get_zs_new(coordinates, kdTree_2D, latent_z, k=50):
    '''
    coordinates: n个要插值的坐标
    kdTree_2D: 降维后的2D坐标
    latent_z: 生成模型的输入潜向量，和kdTree_2D是一一对应关系
    k: 近邻的数量，也就是lagrange插值的节点数量
    '''
    print("最开始的版本，优化了一下代码~~~~~~~~~~~ k:",k)
    # 直接一次查询所有坐标的k个近邻
    nearest_distance, nearest_index = kdTree_2D.query(coordinates, k=k) #这里的k为固定值
    origin_coordinates = kdTree_2D.data # 获取kdtree中原始的坐标
    for iter in range(len(coordinates)):
        iter_distance = nearest_distance[iter]
        iter_index = nearest_index[iter]
        
        sum_distanceForIter = np.sum(iter_distance) #这k个近邻的距离总和
        for i, index in enumerate(iter_index):
            temp_z = torch.tensor(latent_z[index])
            temp_distance = iter_distance[i]
            w = (sum_distanceForIter-temp_distance)/((k-1)*sum_distanceForIter) #对z进行权重
            if i == 0:
                z = temp_z*w
            else:
                z += temp_z*w
        z = z.unsqueeze(0) # shape:[1*latent_dim]
        if iter == 0:
            zs = z
        else:
            zs = torch.cat((zs, z), dim=0)
        # print(zs.shape)
    return zs

# 回归模型的版本，只插值z，不动y(y为最近的那一个)
def get_zs_new_regre(coordinates, kdTree_2D, latent_z, k=2):
    '''
    coordinates: n个要插值的坐标
    kdTree_2D: 降维后的2D坐标
    latent_z: 生成模型的输入潜向量，和kdTree_2D是一一对应关系
    k: 近邻的数量，也就是lagrange插值的节点数量
    '''
    print("最开始的版本，优化了一下代码, 不动y，y只取最近的~~~~~~~~~~~ k:",k)
    # 直接一次查询所有坐标的k个近邻
    nearest_distance, nearest_index = kdTree_2D.query(coordinates, k=k) #这里的k为固定值
    print(nearest_index)
    origin_coordinates = kdTree_2D.data # 获取kdtree中原始的坐标
    for iter in range(len(coordinates)):
        iter_distance = nearest_distance[iter]
        iter_index = nearest_index[iter]
        
        sum_distanceForIter = np.sum(iter_distance) #这k个近邻的距离总和
        # 取最近的那一个y
        label_y_embed = latent_z[iter_index[0]][256:].clone()
        for i, index in enumerate(iter_index):
            temp_z = torch.tensor(latent_z[index])
            temp_distance = iter_distance[i]
            w = (sum_distanceForIter-temp_distance)/((k-1)*sum_distanceForIter) #对z进行权重
            if i == 0:
                z = temp_z[:256]*w
            else:
                z += temp_z[:256]*w
        # print(label_y_embed)
        # print("label_y_embed.shape: ", label_y_embed.shape)
        # print("前z.shape: ", z.shape)
        z = torch.cat((z, label_y_embed))
        # print("后z.shape: ", z.shape)
        z = z.unsqueeze(0) # shape:[1*latent_dim]
        if iter == 0:
            zs = z
        else:
            zs = torch.cat((zs, z), dim=0)
        # print(zs.shape)
    return zs

def get_image_information_other(points, tree_2D, dict_zs, G, DNN_model, Rob_predictor, img_name="one", dataset_type="SteeringAngle"):
    # 根据k个最近邻坐标，计算出坐标对应的z
    print("取z中.....")
    sys.stdout.flush()
    time3 = time.time()
    z = get_zs_new_regre([points], tree_2D, dict_zs)
    print("z.shape: ", z.shape)
    time4 = time.time()
    print("取z消耗时间：", time4-time3)
    sys.stdout.flush()

    print("生成图片中.....")
    sys.stdout.flush()
    time5 = time.time()
    # with torch.no_grad(): # 取消梯度计算，加快运行速度
    z = torch.tensor(z).to(torch.float32).to(device)    # latent code
    
    if dataset_type == "CIFAR10":
        img = G(z) 
        layer = DNN_model(img) #分类模型分类图片
        label = torch.argmax(layer, dim=1)
    elif dataset_type == "SteeringAngle":
        img = G(z[:, :256], z[:, 256:]) 
        # 注册钩子函数到第二层
        DNN_model.pool1.register_forward_hook(get_activation('pool1'))
        label = DNN_model(img) #分类模型分类图片
        layer = activation["pool1"]
        layer = layer.view(layer.size(0), -1)

    img = img/2 + 0.5 # [0,1] 归一化图片的范围到0~1区间
    utils.save_image(img.detach().cpu(), f'./临时垃圾-随时可删/单个坐标生成的图片保存/'+ img_name +'_' + str(points[0]) + ',' + str(point[1]) + '.png')
    # robustness = Rob_predictor(layer) #鲁棒性预测网络预测图片
    # robustness = robustness.detach().cpu()
    # print("robustness.shape: ", robustness.shape)
    image_path = './临时垃圾-随时可删/单个坐标生成的图片保存/'+ img_name + '_' + str(points[0]) + ',' + str(point[1]) +'.png'
    if dataset_type == "CIFAR10":
        robustness = adversarial_robustness(DNN_model, image_path, label, device) # 对抗性攻击算法获得鲁棒性
    elif dataset_type == "SteeringAngle":
        robustness = adversarial_robustness_regre(DNN_model, image_path, label, device) # 对抗性攻击算法获得鲁棒性
    print("robustness: ", robustness)
    sys.stdout.flush()

    time6 = time.time()
    print("生成图片消耗时间：", time6-time5)
    sys.stdout.flush()
    return label.detach().cpu(), robustness, layer.detach().cpu()

In [3]:
dataset_type = "SteeringAngle"
model_name = "ResNet34_regre"
checkpoints_path = "./model_files/SteeringAngle/checkpoints/CcGAN/ckpt_CcGAN_niters_40000_seed_2020_hard.pth"
G = model_all.get_generative_model("SteeringAngle")
G = nn.DataParallel(G, device_ids=[device_id])
G.load_state_dict(torch.load(checkpoints_path, map_location=device)["netG_state_dict"])
G = G.to(device)
G.eval()
# 提前加载预处理的数据（降维后的2维坐标和对应的高维向量）
tree_2D_path="./static/data/SteeringAngle/2D_kdTree/2D_kdTree_200000.pt"
data_z_path="./static/data/SteeringAngle/latent_z/CcGAN_384z_200000.pt"
angles_path="./static/data/SteeringAngle/angle/angles_200000.pt"

tree_2D = torch.load(tree_2D_path)
dict_zs = torch.load(data_z_path, map_location="cpu") #因为我之前保存数据到了GPU上，所以要回到cpu上才不会出错
global angles
angles = torch.load(angles_path)

DNN_model = model_all.get_DNN_model(dataset_type, model_name)
DNN_model.load_state_dict(torch.load("./model_files/" + dataset_type + "/checkpoints/regre_model/" + model_name + ".pt", map_location=device)["net_state_dict"])
DNN_model.eval()
DNN_model.to(device)
Rob_predictor = model_all.get_rob_predictor(dataset_type, model_name)
Rob_predictor.load_state_dict((torch.load("./model_files/"+ dataset_type + "/checkpoints/rob_predictor/kjl_rob_predictor_" + model_name + "_wrongAngel=8.0epsilon=0.001epsilon_step=0.001.pt", map_location=device)))
Rob_predictor.eval()
Rob_predictor.to(device)

Rob_predictor(
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=1, bias=True)
)

In [7]:
point = [-66.215625, -5.775]
point = [-55.190625,-10.434375]
point = [-67.594,-8.531]
point = [-55.05937500000001,-12.271875000000001]
# point = [-39.046875, -14.175000000000004]
img_name = "4"
label, robustness, layer = get_image_information_other(point, tree_2D, dict_zs, G, DNN_model, Rob_predictor, img_name=img_name)


取z中.....
最开始的版本，优化了一下代码, 不动y，y只取最近的~~~~~~~~~~~ k: 2
[[  3644 116979]]
z.shape:  torch.Size([1, 384])
取z消耗时间： 0.0012350082397460938
生成图片中.....
获取对抗鲁棒性中.....


/home/kuangjielong/.conda/envs/python3_7/lib/python3.7/site-packages/ipykernel_launcher.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/kuangjielong/.conda/envs/python3_7/lib/python3.7/site-packages/ipykernel_launcher.py:181: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


误差:  tensor([[0.3221]], device='cuda:0', grad_fn=<AbsBackward>)
获取对抗鲁棒性消耗时间： 1.0263473987579346
robustness:  0.001
生成图片消耗时间： 1.0638799667358398


In [5]:
print((label-0.5)*160)
print(robustness)

tensor([[-64.2508]])
0.005
